<a href="https://colab.research.google.com/github/yn2a75/benkyokai/blob/main/Bert%EF%BC%BFALBERT%E5%AE%9F%E8%A3%85%E3%81%AE%E5%9F%BA%E7%A4%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#環境の準備 

In [2]:
#必要なライブラリのインストール
#fugashi : Mecab の Cython ラッパー
#ipadic : mecab-python3 や fugashi で使用する IPAdic(IPA辞書)

! pip install -q sentencepiece
! pip install -q transformers==4.4.0
! pip install -q fugashi
! pip install -q ipadic

In [3]:
#必要なモジュールのインストール
import torch
import transformers

transformers.__version__


'4.4.0'

#BERTモデルとTokenizerの準備
Tokenizer：必要な分かち書きを行う。事前に用意されているTokenizerモデルを使用する。

In [4]:
from transformers import BertModel, BertJapaneseTokenizer

Bert用Tokenizerのインスタンス化

In [5]:
#分かち書きをするTokenizer
bert_tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

#学習済みBERTのインスタンス化

Hugging Face Transformersで利用できるモデルは様々あり、例えば下記のモデルがある。
- BertModel : 特定のタスクに特化していない、汎用的に使用できるモデル  
- BertFor MaskedLM : 単語の一部に Mask して予測を行うタスクを扱うモデル  
- BertForNextSentencePrediction : ある文章の次の文章が適切かどうかを判断を行うタスクを解くクラス  
- BertForSequencePrediction : 文章分類タスクを行うためのクラス

In [6]:
#日本語学習済みモデル
bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [7]:
#インスタンスしたモデルの構造を可視化
print(bert)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

#Bertの詳細設定を確認
BertConfigクラスを使って使用するBERTモデルの詳細設定を確認

In [8]:
from transformers import BertConfig
#詳細設定の確認
BertConfig.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.4.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

#BERTで分散表現を獲得

In [9]:
text1 = "会社をクビになった。"
text2 = "テレワークばかりでクビが痛い。"
text3 = "会社を解雇になった。"

#Bertの入力を取得

In [10]:
#分かち書きを→id
input_ids1=bert_tokenizer.encode(text1, return_tensors='pt')#pt→Pytorch

In [11]:
#分かち書きの結果を表示。input_ids1にidが格納されている。
print(input_ids1)

tensor([[    2,   811,    11, 13700,     7,    58,    10,     8,     3]])


In [12]:
from transformers.models import bert_japanese
#実際の分かち書きの結果を確認。
print(bert_tokenizer.convert_ids_to_tokens(input_ids1[0]))

['[CLS]', '会社', 'を', 'クビ', 'に', 'なっ', 'た', '。', '[SEP]']


#演習
1. Tokenizer による分かち書きと ID への変換  
2. ID を確認  
3. 分かち書き後のトークン列を表示

In [13]:
#text2
#分かち書き→IDに変換
input_ids2=bert_tokenizer.encode(text2,return_tensors='pt')
#IDの表示
print(input_ids2)
#分かち書き後の単語を表示
print(bert_tokenizer.convert_ids_to_tokens(input_ids2[0]))

tensor([[    2,  5521,  3118,  4027,    12, 13700,    14,  4897, 28457,     8,
             3]])
['[CLS]', 'テレ', '##ワーク', 'ばかり', 'で', 'クビ', 'が', '痛', '##い', '。', '[SEP]']


In [14]:
#text3
#分かち書き→IDに変換
input_ids3=bert_tokenizer.encode(text3,return_tensors='pt')
#IDの表示
print(input_ids3)
#分かち書き後の単語を表示
print(bert_tokenizer.convert_ids_to_tokens(input_ids3[0]))

tensor([[   2,  811,   11, 7279,    7,   58,   10,    8,    3]])
['[CLS]', '会社', 'を', '解雇', 'に', 'なっ', 'た', '。', '[SEP]']


#BERT分散表現を取得（推論）

In [15]:
#%%timeで分散表現にかかる時間を返す
%%time
#モデルに入力
bert.eval()
result1=bert(input_ids1)

CPU times: user 90.1 ms, sys: 12.8 ms, total: 103 ms
Wall time: 209 ms


In [16]:
#BERTの出力は２つないし、３つ。今回は２つ出力される。
#各単語の分散表現は[0]に、[CLS]を更に変換したものが[1]に出力される
#retsult:sequence_output,pooled_output
print(result1[0].shape)
print(result1[1].shape)

torch.Size([1, 9, 768])
torch.Size([1, 768])


#【演習】¶
text2, 3 に対しても行い、それぞれの「クビ」「解雇」の単語の分散表現を取得してください。

In [17]:
%%time
bert.eval()
result2=bert(input_ids2)
result3=bert(input_ids3)

CPU times: user 192 ms, sys: 17.1 ms, total: 209 ms
Wall time: 215 ms


「クビ」、「解雇」の分散表現を取得する


In [18]:
#text1のクビ　3番めのトークン
word_vec1 = result1[0][0][3]

#text2のクビ　5番めのトークン
word_vec2 = result2[0][0][5]

#text3の解雇　3番めのトークン
word_vec3 = result3[0][0][3]


#類似度を測定
類似度をはかる方法としてコサイン類似度を使用する。  
nn.CosineSimilarity()を使用する。

In [19]:
#コサイン類似度を求める
cos =torch.nn.CosineSimilarity(dim=0)

cos_sim_12=cos(word_vec1,word_vec2)
cos_sim_23=cos(word_vec2,word_vec3)
cos_sim_13=cos(word_vec1,word_vec3)

#「クビ（解雇）」と「クビ（首）」の類似度
print(cos_sim_12)
#「クビ（解雇）」と「解雇」の類似度
print(cos_sim_13)

tensor(0.6647, grad_fn=<DivBackward0>)
tensor(0.8597, grad_fn=<DivBackward0>)


#ALBERT実装の基礎
ALBERTでは、NSP(Next sentence prediction )の代わりにSentence Ordering Prediction(SOP) という文章の順番を予測するタスクを用いることで、文の関係性理解を行う。

In [20]:
#必要なライブラリをインストール
from transformers import AlbertTokenizer, AlbertModel

albert_tokenizer = AlbertTokenizer.from_pretrained('ALINEAR/albert-japanese-v2')
albert=AlbertModel.from_pretrained('ALINEAR/albert-japanese-v2')

Downloading:   0%|          | 0.00/785k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.3M [00:00<?, ?B/s]

In [21]:
#設定の確認
from transformers import AlbertConfig

AlbertConfig.from_pretrained('ALINEAR/albert-japanese-v2')

AlbertConfig {
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "eos_token_ids": null,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "output_past": true,
  "pad_token_id": null,
  "position_embedding_type": "absolute",
  "transformers_version": "4.4.0",
  "type_vocab_size": 2,
  "vocab_size": 32000
}

#ALBERTで分散表現を獲得

In [22]:
#text1
#分かち書き→idに変換
input_ids1= albert_tokenizer.encode(text1,return_tensors='pt')
#IDを表示
print(input_ids1)
#分かち書き後の単語を表示
print(albert_tokenizer.convert_ids_to_tokens(input_ids1[0]))

tensor([[    2,    16,   548,    22, 20022,   363,    15,     3]])
['[CLS]', '▁', '会社', 'を', 'クビ', 'になった', '。', '[SEP]']


【演習】それでは、text2, 3 に対しても同様に分かち書きと ID への変換を行ってください。  
1. Tokenizer による分かち書きと ID への変換    
2. ID を確認    
3. 分かち書き後のトークン列を表示  

In [23]:
#text2
#分かち書き→idに変換
input_ids2=albert_tokenizer.encode(text2,return_tensors='pt')
#IDを表示
print(input_ids2)
#分かち書き後の単語を表示
print(albert_tokenizer.convert_ids_to_tokens(input_ids2[0]))


tensor([[    2,    16,  3536,  7045,  3909,    23, 20022,    18,  4154,   127,
            15,     3]])
['[CLS]', '▁', 'テレ', 'ワーク', 'ばかり', 'で', 'クビ', 'が', '痛', 'い', '。', '[SEP]']


In [24]:
#text3
#分かち書き→idに変換
input_ids3=albert_tokenizer.encode(text3,return_tensors='pt')
#IDを表示
print(input_ids3)
#分かち書き後の単語を表示
print(albert_tokenizer.convert_ids_to_tokens(input_ids3[0]))

tensor([[   2,   16,  548,   22, 9529,  363,   15,    3]])
['[CLS]', '▁', '会社', 'を', '解雇', 'になった', '。', '[SEP]']


#ALBERT分散表現を取得

In [25]:
%%time
#モデルに入力
albert.eval()

result1=albert(input_ids1)

CPU times: user 114 ms, sys: 1.52 ms, total: 115 ms
Wall time: 121 ms


In [26]:
print(result1[0].shape)

torch.Size([1, 8, 768])


#【演習】
text2,3 に対しても同様に実装し、合わせて、コサイン類以度を使用して類以度の算出まで行ってください。

In [27]:
result2=albert(input_ids2)
result3=albert(input_ids3)

In [28]:
#単語の分散表現を取得

#text1のクビ　4番目のtoken
word_vec1=result1[0][0][4]

#text2のクビ　6番目のtoken
word_vec2=result2[0][0][6]

#text3の解雇　4番目のtoken
word_vec3=result3[0][0][4]


In [29]:
#コサイン類似度を確かめる
cos=torch.nn.CosineSimilarity(dim=0)

cos_sim12=cos(word_vec1,word_vec2)
cos_sim13=cos(word_vec1,word_vec3)

print(cos_sim12)
print(cos_sim13)

tensor(0.6663, grad_fn=<DivBackward0>)
tensor(0.7592, grad_fn=<DivBackward0>)
